# Training a `Robust' Adapter with AdapterDrop

This notebook extends our quickstart adapter training notebook to illustrate how we can use AdapterDrop
to robustly train an adapter, i.e. adapters that allow us to dynmically dropp layers for faster multi-task inference.
Please have a look at the original adapter training notebook for more details on the setup.

## Installation

First, let's install the required libraries:

In [15]:
!pip install -U adapter-transformers
!pip install datasets

  Cloning https://github.com/hSterz/adapter-transformers.git (to revision notebooks) to /tmp/pip-req-build-9ffnhq9w
  Running command git clone -q https://github.com/hSterz/adapter-transformers.git /tmp/pip-req-build-9ffnhq9w
  Running command git checkout -b notebooks --track origin/notebooks
  Switched to a new branch 'notebooks'
  Branch 'notebooks' set up to track remote branch 'notebooks' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for adapter-transformers: filename=adapter_transformers-2.0.0a1-cp37-none-any.whl size=2009344 sha256=e83ea5be2754c747bd098be28320e515535b097366290d44d0aeb16302f7d1b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-5g7ew8pa/wheels/e2/39/27/3dcf47f4b1bf14923684680dc1f44c2e935944359f16487c86
Successfully built adapter-transformers
  Found existing installation: adapter-transformers 2.0.0a1
    Uninstalling adapter-transformers-2.0.0a1:
      

In [16]:
from datasets import load_dataset
from transformers import RobertaTokenizer

dataset = load_dataset("rotten_tomatoes")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset = dataset.rename_column("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a)
Loading cached processed dataset at /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a/cache-eea56258438b8fa9.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a/cache-f71bd7769a0d3d1b.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a/cache-99fbbc1783d63db9.arrow


## Training

In [17]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"},
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

# Add a new adapter
model.add_adapter("rotten_tomatoes")
# Add a matching classification head
model.add_classification_head("rotten_tomatoes", num_labels=2)
# Activate the adapter
model.train_adapter("rotten_tomatoes")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

To dynamically drop adapter layers during training, we make use of HuggingFace's `TrainerCallback'.

In [18]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction, TrainerCallback

class AdapterDropTrainerCallback(TrainerCallback):
  def on_step_begin(self, args, state, control, **kwargs):
    skip_layers = list(range(np.random.randint(0, 11)))
    kwargs['model'].set_active_adapters("rotten_tomatoes", skip_layers=skip_layers)

  def on_evaluate(self, args, state, control, **kwargs):
    # Deactivate skipping layers during evaluation (otherwise it would use the
    # previous randomly chosen skip_layers and thus yield results not comparable
    # across different epochs)
    kwargs['model'].set_active_adapters("rotten_tomatoes", skip_layers=None)


training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    remove_unused_columns=False
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

trainer.add_callback(AdapterDropTrainerCallback())

We can now train and evaluate our robustly trained adapter!

In [20]:
trainer.train()
trainer.evaluate()

Step,Training Loss
200,0.573800
400,0.362600
600,0.326900
800,0.318700
1000,0.303100
1200,0.293200
1400,0.282300
1600,0.284900


{'epoch': 6.0,
 'eval_acc': 0.8799249530956847,
 'eval_loss': 0.29594820737838745,
 'eval_mem_cpu_alloc_delta': 112111,
 'eval_mem_cpu_peaked_delta': 214624,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 94487040,
 'eval_runtime': 5.0862,
 'eval_samples_per_second': 209.586}